In [1]:
import pandas  as pd
import numpy as np
import tensorflow as tf
import seaborn as sns
%matplotlib inline

2024-03-29 15:51:12.354840: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-29 15:51:12.356249: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-29 15:51:12.403564: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-29 15:51:12.582610: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-29 15:51:13.991819: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF

In [2]:
# Extract the data from the csv files and preperaing for the analysis
DATA_PATHS = [
   '../raw_data/train.csv',
    '../raw_data/test.csv',
    '../raw_data/users.csv',
    '../raw_data/events.csv/events.csv',
    
]
DATA_FRAMES = [pd.read_csv(path) for path in DATA_PATHS]
train_df, test_df, users_df, events_df = DATA_FRAMES

#### TRAIN DATA

In [3]:
# Understanding the data
train_df.head()

,user,event,invited,timestamp,interested,not_interested
0,3044012,1918771225,0,2012-10-02 15:53:05.754000+00:00,0,0
1,3044012,1502284248,0,2012-10-02 15:53:05.754000+00:00,0,0
2,3044012,2529072432,0,2012-10-02 15:53:05.754000+00:00,1,0
3,3044012,3072478280,0,2012-10-02 15:53:05.754000+00:00,0,0
4,3044012,1390707377,0,2012-10-02 15:53:05.754000+00:00,0,0


In [4]:
train_df.describe()

,user,event,invited,interested,not_interested
count,1.539800e+04,1.539800e+04,15398.000000,15398.000000,15398.000000
mean,2.199685e+09,2.060937e+09,0.042473,0.268282,0.033381
std,1.268887e+09,1.190660e+09,0.201672,0.443079,0.179635
min,3.044012e+06,1.040700e+05,0.000000,0.000000,0.000000
25%,1.071319e+09,1.057229e+09,0.000000,0.000000,0.000000
50%,2.259555e+09,1.996503e+09,0.000000,0.000000,0.000000
75%,3.292836e+09,3.060446e+09,0.000000,1.000000,0.000000
max,4.293103e+09,4.294677e+09,1.000000,1.000000,1.000000


In [5]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15398 entries, 0 to 15397
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   user            15398 non-null  int64 
 1   event           15398 non-null  int64 
 2   invited         15398 non-null  int64 
 3   timestamp       15398 non-null  object
 4   interested      15398 non-null  int64 
 5   not_interested  15398 non-null  int64 
dtypes: int64(5), object(1)
memory usage: 721.9+ KB


In [6]:
# No missing values 
train_df.isnull().sum()

user              0
event             0
invited           0
timestamp         0
interested        0
not_interested    0
dtype: int64

#### TEST DATA

In [7]:

test_df.head()

,user,event,invited,timestamp
0,1776192,2877501688,0,2012-11-30 11:39:01.230000+00:00
1,1776192,3025444328,0,2012-11-30 11:39:01.230000+00:00
2,1776192,4078218285,0,2012-11-30 11:39:01.230000+00:00
3,1776192,1024025121,0,2012-11-30 11:39:01.230000+00:00
4,1776192,2972428928,0,2012-11-30 11:39:21.985000+00:00


In [8]:
test_df.describe()

,user,event,invited
count,1.023700e+04,1.023700e+04,10237.000000
mean,2.149160e+09,2.071540e+09,0.039953
std,1.258985e+09,1.192905e+09,0.195859
min,1.776192e+06,1.040700e+05,0.000000
25%,1.038961e+09,1.064770e+09,0.000000
50%,2.151294e+09,2.007279e+09,0.000000
75%,3.269497e+09,3.050793e+09,0.000000
max,4.289725e+09,4.294210e+09,1.000000


#### EVENT DATA

In [9]:
events_df.head()

,event_id,user_id,start_time,city,state,zip,country,lat,lng,c_1,...,c_92,c_93,c_94,c_95,c_96,c_97,c_98,c_99,c_100,c_other
0,684921758,3647864012,2012-10-31T00:00:00.001Z,NaN,NaN,NaN,NaN,NaN,NaN,2,...,0,1,0,0,0,0,0,0,0,9
1,244999119,3476440521,2012-11-03T00:00:00.001Z,NaN,NaN,NaN,NaN,NaN,NaN,2,...,0,0,0,0,0,0,0,0,0,7
2,3928440935,517514445,2012-11-05T00:00:00.001Z,NaN,NaN,NaN,NaN,NaN,NaN,0,...,0,0,0,0,0,0,0,0,0,12
3,2582345152,781585781,2012-10-30T00:00:00.001Z,NaN,NaN,NaN,NaN,NaN,NaN,1,...,0,0,0,0,0,0,0,0,0,8
4,1051165850,1016098580,2012-09-27T00:00:00.001Z,NaN,NaN,NaN,NaN,NaN,NaN,1,...,0,0,0,0,0,0,0,0,0,9


In [10]:
events_df[['event_id','user_id','start_time','city'	,'state','zip','country','lat','lng']].isnull().sum() 


event_id            0
user_id             0
start_time          0
city          1557125
state         1889723
zip           2686286
country       1533009
lat           1331880
lng           1331880
dtype: int64

In [11]:
events_df[['event_id','user_id','start_time','city'	,'state','zip','country','lat','lng']].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3137972 entries, 0 to 3137971
Data columns (total 9 columns):
 #   Column      Dtype  
---  ------      -----  
 0   event_id    int64  
 1   user_id     int64  
 2   start_time  object 
 3   city        object 
 4   state       object 
 5   zip         object 
 6   country     object 
 7   lat         float64
 8   lng         float64
dtypes: float64(2), int64(2), object(5)
memory usage: 215.5+ MB


#### USERS DATA

In [12]:
users_df.head()

,user_id,locale,birthyear,gender,joinedAt,location,timezone
0,3197468391,id_ID,1993,male,2012-10-02T06:40:55.524Z,Medan Indonesia,480.0
1,3537982273,id_ID,1992,male,2012-09-29T18:03:12.111Z,Medan Indonesia,420.0
2,823183725,en_US,1975,male,2012-10-06T03:14:07.149Z,Stratford Ontario,-240.0
3,1872223848,en_US,1991,female,2012-11-04T08:59:43.783Z,Tehran Iran,210.0
4,3429017717,id_ID,1995,female,2012-09-10T16:06:53.132Z,NaN,420.0


In [13]:
users_df.describe()

,user_id,timezone
count,3.820900e+04,37773.000000
mean,2.150982e+09,110.161226
std,1.242134e+09,359.604823
min,6.110000e+03,-720.000000
25%,1.072041e+09,-240.000000
50%,2.153423e+09,240.000000
75%,3.222394e+09,420.000000
max,4.294808e+09,840.000000


In [14]:
users_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38209 entries, 0 to 38208
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   user_id    38209 non-null  int64  
 1   locale     38209 non-null  object 
 2   birthyear  36717 non-null  object 
 3   gender     38100 non-null  object 
 4   joinedAt   38151 non-null  object 
 5   location   32744 non-null  object 
 6   timezone   37773 non-null  float64
dtypes: float64(1), int64(1), object(5)
memory usage: 2.0+ MB


### DATA PREPROCESSING

#### EVENTS AND TRAIN DATA

In [15]:
modified_events_df = events_df.drop(['city','state','zip','country'],axis=1)

In [16]:
modified_events_df.fillna(modified_events_df.mean(),inplace=True)

In [ ]:
modified_events_df.head()

,event_id,user_id,start_time,c_1,c_2,c_3,c_4,c_5,c_6,c_7,...,c_92,c_93,c_94,c_95,c_96,c_97,c_98,c_99,c_100,c_other
0,684921758,3647864012,2012-10-31T00:00:00.001Z,2,0,2,0,0,0,0,...,0,1,0,0,0,0,0,0,0,9
1,244999119,3476440521,2012-11-03T00:00:00.001Z,2,0,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,7
2,3928440935,517514445,2012-11-05T00:00:00.001Z,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,12
3,2582345152,781585781,2012-10-30T00:00:00.001Z,1,0,2,1,0,0,0,...,0,0,0,0,0,0,0,0,0,8
4,1051165850,1016098580,2012-09-27T00:00:00.001Z,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,9


In [ ]:
modified_events_df.isnull() .sum().sum()

0

In [ ]:
train_df['user_id'] = train_df['user']
train_df['event_id'] = train_df['event']

In [ ]:
train_df.head()

,user,event,invited,timestamp,interested,not_interested,user_id,event_id
0,3044012,1918771225,0,2012-10-02 15:53:05.754000+00:00,0,0,3044012,1918771225
1,3044012,1502284248,0,2012-10-02 15:53:05.754000+00:00,0,0,3044012,1502284248
2,3044012,2529072432,0,2012-10-02 15:53:05.754000+00:00,1,0,3044012,2529072432
3,3044012,3072478280,0,2012-10-02 15:53:05.754000+00:00,0,0,3044012,3072478280
4,3044012,1390707377,0,2012-10-02 15:53:05.754000+00:00,0,0,3044012,1390707377


In [ ]:
train_df.drop(['user','event'],axis=1,inplace=True)

In [ ]:
train_df.head()

,invited,timestamp,interested,not_interested,user_id,event_id
0,0,2012-10-02 15:53:05.754000+00:00,0,0,3044012,1918771225
1,0,2012-10-02 15:53:05.754000+00:00,0,0,3044012,1502284248
2,0,2012-10-02 15:53:05.754000+00:00,1,0,3044012,2529072432
3,0,2012-10-02 15:53:05.754000+00:00,0,0,3044012,3072478280
4,0,2012-10-02 15:53:05.754000+00:00,0,0,3044012,1390707377


In [ ]:
import datetime as datetime

train_df['timestamp'] = pd.to_datetime(train_df['timestamp'], format='mixed')

In [ ]:
modified_train_df = train_df.merge(modified_events_df, on=['event_id'], how='left')

In [ ]:
modified_train_df.head()

,invited,timestamp,interested,not_interested,user_id_x,event_id,user_id_y,start_time,c_1,c_2,...,c_92,c_93,c_94,c_95,c_96,c_97,c_98,c_99,c_100,c_other
0,0,2012-10-02 15:53:05.754000+00:00,0,0,3044012,1918771225,4106419938,2012-10-03T08:00:00.002Z,0,0,...,0,0,0,0,0,0,0,0,0,2
1,0,2012-10-02 15:53:05.754000+00:00,0,0,3044012,1502284248,2016654644,2012-10-03T11:00:00.003Z,2,0,...,0,0,0,0,0,0,0,0,0,24
2,0,2012-10-02 15:53:05.754000+00:00,1,0,3044012,2529072432,3639934255,2012-10-26T13:30:00.003Z,2,0,...,0,0,0,1,0,0,0,0,0,37
3,0,2012-10-02 15:53:05.754000+00:00,0,0,3044012,3072478280,97461525,2012-10-06T05:00:00.003Z,1,2,...,0,0,0,0,0,1,0,3,0,56
4,0,2012-10-02 15:53:05.754000+00:00,0,0,3044012,1390707377,3639934255,2012-10-06T03:00:00.003Z,2,0,...,0,0,0,0,0,0,0,1,0,79


In [ ]:
modified_train_df.drop(['user_id_y', 'start_time'],axis=1,inplace=True)

In [ ]:
modified_train_df['user_id'] = modified_train_df['user_id_x']
modified_train_df.drop(['user_id_x'],axis=1,inplace=True)

In [ ]:
modified_train_df.head()

,invited,timestamp,interested,not_interested,event_id,c_1,c_2,c_3,c_4,c_5,...,c_93,c_94,c_95,c_96,c_97,c_98,c_99,c_100,c_other,user_id
0,0,2012-10-02 15:53:05.754000+00:00,0,0,1918771225,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2,3044012
1,0,2012-10-02 15:53:05.754000+00:00,0,0,1502284248,2,0,0,0,2,...,0,0,0,0,0,0,0,0,24,3044012
2,0,2012-10-02 15:53:05.754000+00:00,1,0,2529072432,2,0,0,0,0,...,0,0,1,0,0,0,0,0,37,3044012
3,0,2012-10-02 15:53:05.754000+00:00,0,0,3072478280,1,2,1,1,0,...,0,0,0,0,1,0,3,0,56,3044012
4,0,2012-10-02 15:53:05.754000+00:00,0,0,1390707377,2,0,0,0,0,...,0,0,0,0,0,0,1,0,79,3044012
